In [10]:
import tkinter as tk
import tkinter.ttk
import pandas as pd # import pandas library 
from tkinter import messagebox

In [11]:
#creating tkinter window object-----------------------------------
root=tk.Tk()
root.title("Movie Recommender")
root.geometry('800x400')

#getting_data_ready------------------------------------------------------------------
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
path = 'user.tsv'
df = pd.read_csv(path, sep='\t', names=column_names)

movie_titles = pd.read_csv('Movie_Id_Titles')
data = pd.merge(df, movie_titles, on='item_id')

ratings = pd.DataFrame(data.groupby('title')['rating'].mean())
ratings['num of ratings'] = pd.DataFrame(data.groupby('title')['rating'].count())

moviemat = data.pivot_table(index ='user_id', 
              columns ='title', values ='rating') 

#Predicting----------------------------------------------------------------
def Analyse():
    res=C1.get()
    if(res!='<Select a Movie>'):
        movie_user_ratings = moviemat[C1.get()] 
        similar_to_movie = moviemat.corrwith(movie_user_ratings) 
        corr_movie = pd.DataFrame(similar_to_movie, columns =['Correlation']) 
        corr_movie.dropna(inplace = True)
        corr_movie.sort_values('Correlation', ascending = False)
        corr_movie = corr_movie.join(ratings['num of ratings']) 
        #print(corr_movie.head())
        result=corr_movie[corr_movie['num of ratings']>100][corr_movie['Correlation']>=0.5].sort_values('Correlation', ascending = False).head() 
        #print(result)
        T1.config(state='normal')
        T1.delete('1.0',tk.END)
        T1.insert('1.0',result)
        T1.config(state='disabled')

        messagebox.showinfo('Report','Analysis Complete')

    else:
        messagebox.showinfo('Alert!','Please select a movie')
        
#setting widgets in tkinter window-----------------------------------------------------------------
L1=tk.Label(root,text="Movie Recommender",font=('bold',20))
L1.pack()
L1.place(x=10,y=5)

movielist=list(movie_titles['title'])
movielist.sort()
C1=tk.ttk.Combobox(root,value=movielist,width=50,state="readonly")
C1.set('<Select a Movie>')
C1.pack()
C1.place(x=40,y=52)

B1=tk.Button(root,text='Analyse',command=Analyse,bd=5)
B1.pack()
B1.place(x=380,y=47)

T1=tk.Text(root,height=18,width=96,state='disabled',bd=5)
T1.pack()
T1.place(x=10,y=90)

#running the tkinter window-------------------------------------------------------------------
root.resizable(width=False,height=False)
root.mainloop()